# CS6140 Assignment 3: Regression Models Part 1
- Student: Sukhrobbek Ilyosbekov

## Setup
First of all, we import all common used libraries and load CSV file into a pandas dataframe.

In [1]:
import os
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Add the parent directory to the Python path to import the core module
sys.path.append(os.path.abspath(os.path.join("..")))

from core import get_data_path

csv_file_path = get_data_path("assignment2/lc_14to16.csv")
data = pd.read_csv(csv_file_path)

C:\Users\suxro\AppData\Local\Temp\ipykernel_16500\2447354417.py:13: DtypeWarning: Columns (19,59,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv_file_path)
